In [ ]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 
import fileinput
import re

In [ ]:
rootdir="../Kaggle/pre-data"
backup = "../Kaggle/backup" # place holder for backup data


In [ ]:
"""
 Kaggle

step 1: Extracting txt file content and preparing trans.txt for each main folders.
step 2: Converting.
step 3: Checking the wav audio specifications.
step 4: Renaming files.
step 5: Cleaning folders.

"""

  
# (a) Extracting txt file content and preparing trans.txt for each main folders
if True: 
    for file in natsort.natsorted(os.listdir(rootdir)):
        if(file.endswith(".txt")):
            a_Filename = []
            a_text = []
            a_time = []
            #-------------------
            m_Filename = []
            m_text = []
            m_time = []
            #-------------------
            s_Filename = []
            s_text = []
            s_time = []
            with codecs.open(rootdir+"/"+file, 'r', encoding='utf8') as f:
                line = f.readline()
                cnt = 1
                while line:
                    each_line = line.strip()
                    each_line_split = each_line.split('|')
                    if(each_line_split[0].split('/')[0] == "achtgesichterambiwasse"):
                        a_Filename.append(each_line_split[0].split('/')[-1].split('.')[0])
                        a_text.append(each_line_split[-2].upper())
                        a_time.append(each_line_split[-1])                
                    if(each_line_split[0].split('/')[0] == "meisterfloh"):
                        m_Filename.append(each_line_split[0].split('/')[-1].split('.')[0])
                        m_text.append(each_line_split[-2].upper())
                        m_time.append(each_line_split[-1])
                    if(each_line_split[0].split('/')[0] == "serapionsbruederauswahl"):
                        s_Filename.append(each_line_split[0].split('/')[-1].split('.')[0])
                        s_text.append(each_line_split[-2].upper())
                        s_time.append(each_line_split[-1])
                    line = f.readline()  
                    cnt += 1
                df_a = pd.DataFrame({'col1':a_Filename, 'col2':a_text, 'col3':a_time})
                df_m = pd.DataFrame({'col1':m_Filename, 'col2':m_text, 'col3':m_time})
                df_s = pd.DataFrame({'col1':s_Filename, 'col2':s_text, 'col3':s_time})
                with codecs.open(rootdir+"/achtgesichterambiwasse/"+"a.trans.txt", "a", encoding='utf8') as trans_file_a:
                    for i in range(len(df_a)):
                        trans_file_a.write(df_a['col1'][i]+" "+df_a['col2'][i]+"\n")
                with codecs.open(rootdir+"/meisterfloh/"+"m.trans.txt", "a", encoding='utf8') as trans_file_m:
                    for i in range(len(df_m)):
                        trans_file_m.write(df_m['col1'][i]+" "+df_m['col2'][i]+"\n")
                with codecs.open(rootdir+"/serapionsbruederauswahl/"+"s.trans.txt", "a", encoding='utf8') as trans_file_s:
                    for i in range(len(df_s)):
                        trans_file_s.write(df_s['col1'][i]+" "+df_s['col2'][i]+"\n")
# (b) Converting
if True: 
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if(not Dir.endswith(".txt")):
            if not os.path.exists(rootdir+"/"+Dir+"/temp"): 
                os.makedirs(rootdir+"/"+Dir+"/temp")
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
                if(file.endswith(".wav")):
                    actual_filename =file[:-4]
                    a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp", actual_filename))# (source,target(half path),name_of_file)
                    print(a)
                    if(a != 0):
                            print("Did not get converted")

# (c) Checking the wav audio specifications
if True:
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if(not Dir.endswith(".txt")):
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
                if(file.endswith(".wav")):
                    f = wave.open(rootdir+"/"+Dir+"/temp"+"/"+file, "rb")
                    print(f.getnchannels()) #Returns number of audio channels (1 for mono, 2 for stereo).                    
                    print(f.getframerate()) #Returns sampling frequency.
                    
# (d) Renaming
if True: 
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if(not(Dir.endswith(".txt"))):
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
                if(file.endswith(".wav")):
                    os.rename(rootdir+"/"+Dir+"/temp/"+file,rootdir+"/"+Dir+"/temp/"+file[:-4]+".flac")
                    
# (e) Cleaning folders
if True: 
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if(not(Dir.endswith(".txt"))):
            for file1 in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
                if(file1.endswith(".wav")):
                    os.remove(rootdir+"/"+Dir+"/"+file1)
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
                shutil.move(rootdir+"/"+Dir+"/temp/"+file, rootdir+"/"+Dir+"/"+file)
            os.rmdir(rootdir+"/"+Dir+"/temp/")
        if(Dir.endswith(".txt")):
            os.remove(rootdir+"/"+Dir)
            
